In [29]:
import pandas as pd
import pickle5 as pickle

selected_stanza_ents = pd.read_csv('../../data/our-wikipedia-corpus/Tokens_From_Question_side/mini_corpus-10T5CpT.csv')
stanza_ents_main = pd.read_pickle(open('../../data/stanza_ents-from_context.pkl','rb'))

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model_vocab = list(tokenizer.vocab.keys())

In [30]:
from tqdm.auto import tqdm

#Removing those entities that are also in the model vocab since we don't want 2 of the same token embedding.
ent_in_model_vocab = []
for ent in tqdm(stanza_ents_main):
    if ent in model_vocab:
        ent_in_model_vocab.append(ent)

for ent in ent_in_model_vocab:
    stanza_ents_main.remove(ent)

#Removing all occurrences of the selected entities. This is where we want fresh new entities & contexts.
for ent in list(set(selected_stanza_ents.ent.to_list())):
    if ent in stanza_ents_main:
        stanza_ents_main = list(filter((ent).__ne__, stanza_ents_main))

  0%|          | 0/92265 [00:00<?, ?it/s]

In [31]:
from collections import Counter
s = Counter(stanza_ents_main)
sorted_ent_counts = dict(sorted(s.items(), key=lambda item: item[1], reverse=True))

In [32]:
keys_to_remove = []
for key in sorted_ent_counts.keys():
    if len(key) == 1:
        keys_to_remove.append(key)
        
for key in keys_to_remove:
    sorted_ent_counts.pop(key)

In [ ]:
#With only one context approach - most likely the one to use

import wikipedia
from collections import defaultdict

no_of_ents_to_select = 100
selected_ents_text_dict = defaultdict(list)

for ent in tqdm(sorted_ent_counts.keys()):
    if len(selected_ents_text_dict.keys()) == no_of_ents_to_select:
        break

    try:
        search_result = wikipedia.search(str(ent), results=1)[0]
    except:
        continue

    try:
        wiki_query = wikipedia.page(search_result, auto_suggest=False)
        selected_ents_text_dict[ent].append(wiki_query.summary)
        selected_ents_text_dict[ent].append(wiki_query.content)
    except:
        continue
        
    if len(selected_ents_text_dict.keys()) % 100 == 0:
        print(f'No. of entities selected so far: {len(selected_ents_text_dict.keys())}') 

In [36]:
import wikipedia
from collections import defaultdict

no_of_ents_to_select = 50
no_of_results_per_entity = 2

selected_ents_text_dict = defaultdict(list)

for ent in tqdm(sorted_ent_counts.keys()):
    if len(selected_ents_text_dict) == no_of_ents_to_select:
        break
    
    search_results = wikipedia.search(str(ent), results=no_of_results_per_entity)
    
    no_of_ctx_available = 0
    final_ctx = ''
    
    for res in search_results:
        try:
            final_ctx += wikipedia.page(res, auto_suggest=False).content
            no_of_ctx_available += 1          
        except:
            continue
    
    if no_of_ctx_available == no_of_results_per_entity:
        selected_ents_text_dict[ent].append(wikipedia.page(search_results[0], auto_suggest=False).summary)
        selected_ents_text_dict[ent].append(final_ctx)
        
print(f'Entities that were selected: {selected_ents_text_dict.keys()}')

  0%|          | 0/29484 [00:00<?, ?it/s]

/home/saptarshi/anaconda3/lib/python3.9/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/saptarshi/anaconda3/lib/python3.9/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Entities that were selected: dict_keys(['DNA', 'MERS-CoV', 'HTNV', 'H5N1', 'IFITM5', 'E. coli', 'IFN', 'PCR', 'MKP-1', 'COVID', 'Spo0A', 'B cells', 'respiratory viruses', 'nucleotide', 'IFITM3', 'CC10', 'NEAT1', 'amino acid', 'our study', 'influenza virus', 'IgG', 'IAV', 'the disease', 'HUVECs', 'macrophages', 'influenza viruses', 'EGR1', 'SARS-CoV', 'RIG-I', 'cytokine', 'Ebola', 'CEACAM1', 'mucosal', 'rVSV', 'Fgl2', 'C. difficile', 'nucleolar', 'HIV', 'HMPV', 'CD40L', 'Zanamivir', 'PBS', 'PEDV', 'nucleolus', 'pertussis', 'consistent with', 'intracellular', 'IgM', 'B cell', 'coronaviruses'])


In [37]:
sample_id = []
title = []
context = []
question = []
answers = []

for idx, (ent, page_data) in enumerate(selected_ents_text_dict.items()):
    sample_id.append(str(idx))
    title.append(ent)
    context.append(page_data[1])
    question.append(f'What is {ent}?')
    answers.append({'text': [page_data[0]], 'answer_start': [page_data[1].find(page_data[0])]})
    
#Saving the dataframe as parquet since it was messing up formats.
pd.DataFrame(zip(sample_id, title, context, question, answers), 
             columns = ['id', 'title', 'context', 'question'
                       ,'answers']).to_parquet('mini-corpus-for-extended-QA')

In [38]:
from datasets import load_dataset
import os

s = load_dataset('parquet', data_files=os.path.abspath('../../data/our-wikipedia-corpus/mini-corpus-for-extended-QA'))

Using custom data configuration default-4fbb762fb5bcbb9f


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /home/saptarshi/.cache/huggingface/datasets/parquet/default-4fbb762fb5bcbb9f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [45]:
s['train'][1]['context']

'Middle East respiratory syndrome–related coronavirus (MERS-CoV), or EMC/2012 (HCoV-EMC/2012), is the virus that causes Middle East respiratory syndrome (MERS). It is a species of coronavirus which infects humans, bats, and camels. The infecting virus is an enveloped, positive-sense, single-stranded RNA virus which enters its host cell by binding to the DPP4 receptor. The species is a member of the genus Betacoronavirus and subgenus Merbecovirus.Initially called simply novel coronavirus or nCoV, it was first reported in June 2012 after genome sequencing of a virus isolated from sputum samples from a person who fell ill in a 2012 outbreak of a new flu-like respiratory illness. By July 2015, MERS-CoV cases had been reported in over 21 countries, in Europe, North America and Asia as well as the Middle East. MERS-CoV is one of several viruses identified by the World Health Organization (WHO) as a likely cause of a future epidemic. They list it for urgent research and development.\n\n\n== V

In [44]:
wikipedia.search('DNA')

['DNA',
 'DNA–DNA hybridization',
 'DNA²',
 'Mitochondrial DNA',
 'DNA methylation',
 'A-DNA',
 'Circular DNA',
 'DNA profiling',
 'DnaA',
 'DNA replication']